In [3]:


import os


import random
import numpy as np
import torch
import cv2
import timm
import pandas as pd
import copy
import matplotlib.pyplot as plt
import torchvision.transforms as transform
from sklearn import model_selection
from torch.utils.data import Dataset
import torch.nn as nn
import torch.optim as optim

from torch.optim.lr_scheduler import StepLR, CosineAnnealingWarmRestarts, CosineAnnealingLR
from torch.utils.data import DataLoader
import models.create_models as create
from tqdm import tqdm

from dataset import CustomDataset, MultiviewImgDataset, SingleimgDataset
from models.FLoss import FocalLoss
from sklearn.model_selection import train_test_split
from ml_decoder import MLDecoder
#cyn
import argparse
import datetime
import json

import time
from pathlib import Path

/home/cyn/anaconda3/envs/cyn1/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df_gt_1 = pd.read_excel('/data/cyn/code/MVCINN/MVCINN-code/医生验证结果（1）.xlsx')
df_gt_2 = pd.read_excel('/data/cyn/code/MVCINN/MVCINN-code/医生验证结果（2）.xlsx')
df_gt_3 = pd.read_excel('/data/cyn/code/MVCINN/MVCINN-code/医生验证结果（3）.xlsx')
df_pred=pd.read_excel('/data/cyn/code/MVCINN/MVCINN-code/MVCINN_test_Nfnet_conv2d_SKnet_True_M_4_r8_L_32_is_weighted_joint_False_fuse_view_False_concat_False_ablation_fusion_False_using_decoder_True_decoder_embedding_3072_num_layers_decoder_8_best_acc_交叉验证.xlsx')
df_gt_1 =df_gt_1.iloc[:250]
df_gt_2 =df_gt_2.iloc[:250]
df_gt_3 =df_gt_3.iloc[:250]
df_pred=df_pred.iloc[:250]
df_test=pd.read_csv('/data/cyn/data/MFIDDR/test_fourpic_label.csv')
df_test=df_test.iloc[:250]

In [5]:
df_test.level.values

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0])

In [6]:
df_gt_1['诊断结果（0-4）'].values,df_gt_1['诊断结果（0-4）.1'].values,df_gt_1['诊断结果（0-4）.2'].values,df_gt_1['诊断结果（0-4）.3'].values

(array(['N,0', 'N,0', 'N,0', 'N,0', 'N,0', 'N,0', 'N,0', 'N,0', 'N,0',
        'N,0', 'N,0', 'N,0', 'N,0', 'N,0', 'N,0', 'N,0', 'N,0', 'N,0',
        'N,0', 'N,0', 'N,0', 'N,0', 'N,0', 'N,0', 'N,0', 'N,0', 'N,0',
        'N,0', 'N,0', 'N,0', 'N,0', 'N,0', 'N,0', 'N,0', 'N,0', 'N,0',
        'N,0', 'N,0', 'N,0', 'N,0', 'N,0', 'N,0', 'Ⅰ,1', 'N,0', 'N,0',
        'N,0', 'N,0', 'N,0', 'N,0', 'N,0', 'N,0', 'N,0', 'N,0', 'N,0',
        'N,0', 'Ⅰ,1', 'N,0', 'N,0', 'N,0', 'Ⅲ,2', 'N,0', 'N,0', 'N,0',
        'N,0', 'N,0', 'N,0', 'N,0', 'N,0', 'N,0', 'N,0', 'N,0', 'N,0',
        'N,0', 'N,0', 'N,0', 'N,0', 'N,0', 'N,0', 'N,0', 'N,0', 'N,0',
        'N,0', 'N,0', 'N,0', 'N,0', 'N,0', 'N,0', 'N,0', 'N,0', 'N,0',
        'N,0', 'N,0', 'N,0', 'N,0', 'Ⅰ,1', 'N,0', 'N,0', 'N,0', 'N,0',
        'N,0', 'N,0', 'N,0', 'N,0', 'N,0', 'N,0', 'N,0', 'N,0', 'N,0',
        'N,0', 'N,0', 'N,0', 'N,0', 'N,0', 'N,0', 'N,0', 'N,0', 'N,0',
        'N,0', 'N,0', 'N,0', 'N,0', 'N,0', 'N,0', 'N,0', 'N,0', 'N,0',
      

In [7]:
df_pred['分级结果'].values

array([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0])

In [8]:
df_gt_3['诊断结果（0-4）'].value_counts(),df_gt_3['诊断结果（0-4）.1'].value_counts(),df_gt_3['诊断结果（0-4）.2'].value_counts(),df_gt_3['诊断结果（0-4）.3'].value_counts(),df_pred['分级结果'].value_counts()

(诊断结果（0-4）
 0.0    243
 1.0      7
 Name: count, dtype: int64,
 诊断结果（0-4）.1
 0.0    244
 1.0      6
 Name: count, dtype: int64,
 诊断结果（0-4）.2
 0.0    244
 1.0      6
 Name: count, dtype: int64,
 诊断结果（0-4）.3
 0.0    245
 1.0      5
 Name: count, dtype: int64,
 分级结果
 0    229
 1     21
 Name: count, dtype: int64)

In [9]:
pred=np.append(df_pred['分级结果'].values,df_pred['分级结果'].values)
pred=np.append(pred,pred)

In [10]:
gt_1=np.array([])
for item in df_gt_1['诊断结果（0-4）'].values:
    if item=='N,0':
        gt_1 = np.append(gt_1, 0)
    elif item=='Ⅰ,1':
        gt_1 = np.append(gt_1, 1)
    elif item=='Ⅲ,2':
        gt_1 = np.append(gt_1, 2)
for item in df_gt_1['诊断结果（0-4）.1'].values:
    if item=='N,0':
        gt_1 = np.append(gt_1, 0)
    elif item=='Ⅰ,1':
        gt_1 = np.append(gt_1, 1)
    elif item=='Ⅲ,2':
        gt_1 = np.append(gt_1, 2)      
for item in df_gt_1['诊断结果（0-4）.2'].values:
    if item=='N,0':
        gt_1 = np.append(gt_1, 0)
    elif item=='Ⅰ,1':
        gt_1 = np.append(gt_1, 1)
    elif item=='Ⅲ,2':
        gt_1 = np.append(gt_1, 2)      
for item in df_gt_1['诊断结果（0-4）.3'].values:
    if item=='N,0':
        gt_1 = np.append(gt_1, 0)
    elif item=='Ⅰ,1':
        gt_1 = np.append(gt_1, 1)
    elif item=='Ⅲ,2':
        gt_1 = np.append(gt_1, 2)    
        


In [11]:
gt_2=np.array([])
for item in df_gt_2['诊断结果（0-4）'].values:
    if item==0.0:
        gt_2 = np.append(gt_2, 0)
    elif item==1.0:
        gt_2 = np.append(gt_2, 1)
    elif item==2.0:
        gt_2 = np.append(gt_2, 2)
for item in df_gt_2['诊断结果（0-4）.1'].values:
    if item==0.0:
        gt_2 = np.append(gt_2, 0)
    elif item==1.0:
        gt_2 = np.append(gt_2, 1)
    elif item==2.0:
        gt_2 = np.append(gt_2, 2)      
for item in df_gt_2['诊断结果（0-4）.2'].values:
    if item==0.0:
        gt_2 = np.append(gt_2, 0)
    elif item==1.0:
        gt_2 = np.append(gt_2, 1)
    elif item==2.0:
        gt_2 = np.append(gt_2, 2)      
for item in df_gt_2['诊断结果（0-4）.3'].values:
    if item==0.0:
        gt_2 = np.append(gt_2, 0)
    elif item==1.0:
        gt_2 = np.append(gt_2, 1)
    elif item==2.0:
        gt_2 = np.append(gt_2, 2)    

In [12]:
gt_3=np.array([])
for item in df_gt_3['诊断结果（0-4）'].values:
    if item==0.0:
        gt_3 = np.append(gt_3, 0)
    elif item==1.0:
        gt_3 = np.append(gt_3, 1)
    elif item==2.0:
        gt_3 = np.append(gt_3, 2)
for item in df_gt_3['诊断结果（0-4）.1'].values:
    if item==0.0:
        gt_3 = np.append(gt_3, 0)
    elif item==1.0:
        gt_3 = np.append(gt_3, 1)
    elif item==2.0:
        gt_3 = np.append(gt_3, 2)      
for item in df_gt_3['诊断结果（0-4）.2'].values:
    if item==0.0:
        gt_3 = np.append(gt_3, 0)
    elif item==1.0:
        gt_3 = np.append(gt_3, 1)
    elif item==2.0:
        gt_3 = np.append(gt_3, 2)      
for item in df_gt_3['诊断结果（0-4）.3'].values:
    if item==0.0:
        gt_3 = np.append(gt_3, 0)
    elif item==1.0:
        gt_3 = np.append(gt_3, 1)
    elif item==2.0:
        gt_3 = np.append(gt_3, 2)    

In [13]:
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import accuracy_score

In [14]:
kappa_1 = cohen_kappa_score(gt_1, pred)
kappa_2 = cohen_kappa_score(gt_2, pred)
kappa_3 = cohen_kappa_score(gt_3, pred)

In [15]:
kappa_1,kappa_2,kappa_3

(0.4339495116758474, 0.6074326092645904, 0.42289935364727604)

In [16]:
accuracy_1 = accuracy_score(gt_1, pred)
accuracy_2 = accuracy_score(gt_2, pred)
accuracy_3 = accuracy_score(gt_3, pred)

In [17]:
accuracy_1,accuracy_2,accuracy_3

(0.933, 0.952, 0.94)

In [24]:
from scipy.stats import pearsonr
print(pearsonr(gt_1, gt_2))
print(pearsonr(gt_2, gt_3))
print(pearsonr(gt_1, gt_3))

PearsonRResult(statistic=0.47850401670550696, pvalue=2.3474414796156362e-58)
PearsonRResult(statistic=0.7141278717770038, pvalue=9.233671458087035e-157)
PearsonRResult(statistic=0.5645874236209011, pvalue=2.9289551302567306e-85)


In [23]:
correlation_coefficient,p_value

(0.47850401670550696, 2.3474414796156362e-58)

In [19]:
accuracy_4=accuracy_score(df_test.level.values,pred[:250])

In [20]:
accuracy_4

0.964